In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import requests as rq
from io import BytesIO
import zipfile
import xmltodict
import json
import pandas as pd
from sqlalchemy import create_engine
import pymysql
from datetime import date
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
import os

In [5]:
pip show tqdm

Name: tqdm
Version: 4.65.0
Summary: Fast, Extensible Progress Meter
Home-page: https://tqdm.github.io
Author: 
Author-email: 
License: MPLv2.0, MIT Licences
Location: /opt/homebrew/Caskroom/miniconda/base/envs/yeardream/lib/python3.9/site-packages
Requires: 
Required-by: finance-datareader, huggingface-hub, nltk, openai, optuna
Note: you may need to restart the kernel to use updated packages.


In [37]:
predicted_data_combined


,Date,Predicted Close Price,stock_code,stock_name
0,2023-09-21,4413.608254,095570,AJ네트웍스
1,2023-09-21,19115.458024,006840,AK홀딩스
2,2023-09-21,3665.154136,027410,BGF
3,2023-09-21,148953.601414,282330,BGF리테일
4,2023-09-21,6988.950019,138930,BNK금융지주
...,...,...,...,...
2317,2023-09-21,7473.347520,024060,흥구석유
2318,2023-09-21,6457.745076,010240,흥국
2319,2023-09-21,2275.888796,189980,흥국에프엔비
2320,2023-09-21,8685.170650,037440,희림


In [ ]:
ip =  '52.79.207.181'
engine = create_engine(f'mysql+pymysql://jun:1234qwer@{ip}:3306/quant')
con = pymysql.connect(user='jun',
                      passwd='1234qwer',
                      host= ip,
                      db='quant',
                      charset='utf8')
mycursor = con.cursor()
# read the sql 
sql_query = """
SELECT * FROM df
"""

# con을 사용하여 데이터를 읽음
stock_price = pd.read_sql_query(sql_query, con)


unique_stock_codes = stock_price['stock_code'].unique()
predicted_data_list = []  # 빈 리스트 생성
failed_stock_codes = []


for stock_code in unique_stock_codes:
    # 특정 stock_code에 해당하는 데이터만 추출
    stock_data = stock_price[stock_price['stock_code'] == stock_code]
    # 예측한 날짜 범위 생성 
    predicted_dates = pd.date_range(start=stock_data['date'].iloc[-1] + pd.Timedelta(days=1), periods=1)

    if len(stock_data) >= seq_len:
        metadata = stock_data[['stock_code', 'stock_name']].drop_duplicates()
        stock_data.drop(['index', 'stock_code', 'stock_name'], axis=1, inplace=True) # delete 'index', 'stock_code', 'stock_name'
        try:
            # 저장할 폴더 경로
            save_folder = './save_weights/'

            # 폴더가 존재하지 않으면 생성
            if not os.path.exists(save_folder):
                os.makedirs(save_folder)

            # save original 'open' prices for later
            original_open = stock_data['open'].values

            # separate dates for future plotting
            dates = pd.to_datetime(stock_data['date'])

            # variables for training
            cols = list(stock_data)[1:6]

            # new dataframe with only training data - 5 columns
            stock_data = stock_data[cols].astype(float)

            # normalize the dataset
            scaler = StandardScaler()
            scaler = scaler.fit(stock_data)
            stock_data_scaled = scaler.transform(stock_data)

            # split to train data and test data
            n_train = int(0.9*stock_data_scaled.shape[0])
            train_data_scaled = stock_data_scaled[0: n_train]
            train_dates = dates[0: n_train]

            test_data_scaled = stock_data_scaled[n_train:]
            test_dates = dates[n_train:]
            # print(test_dates.head(5))

            # data reformatting for LSTM
            pred_days = 1  # prediction period
            seq_len = 14   # sequence length = past days for future prediction.
            input_dim = 5  # input_dimension = ['open', 'high', 'low', 'close', 'volume']

            trainX = []
            trainY = []
            testX = []
            testY = []

            for i in range(seq_len, n_train-pred_days +1):
                trainX.append(train_data_scaled[i - seq_len:i, 0:train_data_scaled.shape[1]])
                trainY.append(train_data_scaled[i + pred_days - 1:i + pred_days, 0])

            for i in range(seq_len, len(test_data_scaled)-pred_days +1):
                testX.append(test_data_scaled[i - seq_len:i, 0:test_data_scaled.shape[1]])
                testY.append(test_data_scaled[i + pred_days - 1:i + pred_days, 0])

            trainX, trainY = np.array(trainX), np.array(trainY)
            testX, testY = np.array(testX), np.array(testY)

            # print(trainX.shape, trainY.shape)
            # print(testX.shape, testY.shape)

            # LSTM model
            model = Sequential()
            model.add(LSTM(64, input_shape=(trainX.shape[1], trainX.shape[2]), # (seq length, input dimension)
                        return_sequences=True))
            model.add(LSTM(32, return_sequences=False))
            model.add(Dense(trainY.shape[1]))

            model.summary()

            # specify your learning rate
            learning_rate = 0.01
            # create an Adam optimizer with the specified learning rate
            optimizer = Adam(learning_rate=learning_rate)
            # compile your model using the custom optimizer
            model.compile(optimizer=optimizer, loss='mse')


            # Try to load weights
            try:
                model.load_weights('./save_weights/lstm_weights.h5')
                
                print("Loaded model weights from disk")
            except:
                print("No weights found, training model from scratch")
                # Fit the model
                history = model.fit(trainX, trainY, epochs=30, batch_size=32,
                                validation_split=0.1, verbose=1)
                # Save model weights after training
                model.save_weights('./save_weights/lstm_weights.h5')

                plt.plot(history.history['loss'], label='Training loss')
                plt.plot(history.history['val_loss'], label='Validation loss')
                plt.legend()
                plt.show()


            # prediction
            prediction = model.predict(testX)

            # generate array filled with means for prediction
            mean_values_pred = np.repeat(scaler.mean_[np.newaxis, :], prediction.shape[0], axis=0)
            mean_values_pred[:, 0] = np.squeeze(prediction)
            y_pred = scaler.inverse_transform(mean_values_pred)[:, 0]

            # generate array filled with means for testY
            mean_values_testY = np.repeat(scaler.mean_[np.newaxis, :], testY.shape[0], axis=0)
            mean_values_testY[:, 0] = np.squeeze(testY)
            testY_original = scaler.inverse_transform(mean_values_testY)[:, 0]

            # 새로운 테스트 데이터 생성 (2023-09-21일을 포함하여 마지막 14일의 데이터)
            new_test_data = stock_data_scaled[-seq_len:].reshape(1, seq_len, input_dim)

            # 새로운 테스트 데이터로 예측 수행
            new_prediction = model.predict(new_test_data)

            # 역변환하여 원래의 주가 단위로 되돌리기
            mean_values_pred = np.repeat(scaler.mean_[np.newaxis, :], new_prediction.shape[0], axis=0)
            mean_values_pred[:, 0] = np.squeeze(new_prediction)
            predicted_close_price = scaler.inverse_transform(mean_values_pred)[:, 0]



            # 예측 결과를 데이터 프레임으로 저장
            predicted_data = pd.DataFrame({
                'Date': predicted_dates,
                'Predicted Close Price': predicted_close_price,
                'stock_code': metadata['stock_code'].values[0],
                'stock_name': metadata['stock_name'].values[0]
            })
            predicted_data_list.append(predicted_data)  # 리스트에 예측 데이터 추가

        except Exception as e:
            print(f"Error processing stock_code {stock_code}: {e}")
            failed_stock_codes.append(stock_code)


# 리스트에 있는 데이터를 하나의 DataFrame으로 합치기
predicted_data_combined = pd.concat(predicted_data_list, ignore_index=True)

